In [1]:
import math
import json
import numpy as np
import pandas as pd
import re
from copy import copy
from tqdm import tqdm
import df_utils
from treys import Card, Evaluator
from numbers import Number
from itertools import chain

### Reading the data

In [2]:
df = pd.read_csv('data/holdem3.csv', low_memory=False)

In [3]:
pd.set_option('display.max_columns', None)
df

,game_id,dealer_id,players_num,year,month,timestamp,board_card_1,board_card_2,board_card_3,board_card_4,board_card_5,player1_total_bet,player1_bankroll,player1_total_win,player1_bet_preflop,player1_bet_flop,player1_bet_turn,player1_bet_river,player1_pocket_card1,player1_pocket_card2,player1_name,player2_total_bet,player2_bankroll,player2_total_win,player2_bet_preflop,player2_bet_flop,player2_bet_turn,player2_bet_river,player2_pocket_card1,player2_pocket_card2,player2_name,pot_players_num_flop,pot_size_flop,pot_players_num_turn,pot_size_turn,pot_players_num_river,pot_size_river,pot_players_num_showdown,pot_size_showdown,player3_total_bet,player3_bankroll,player3_total_win,player3_bet_preflop,player3_bet_flop,player3_bet_turn,player3_bet_river,player3_pocket_card1,player3_pocket_card2,player3_name,player4_total_bet,player4_bankroll,player4_total_win,player4_bet_preflop,player4_bet_flop,player4_bet_turn,player4_bet_river,player4_pocket_card1,player4_pocket_card2,player4_name,player5_total_bet,player5_bankroll,player5_total_win,player5_bet_preflop,player5_bet_flop,player5_bet_turn,player5_bet_river,player5_pocket_card1,player5_pocket_card2,player5_name,player6_total_bet,player6_bankroll,player6_total_win,player6_bet_preflop,player6_bet_flop,player6_bet_turn,player6_bet_river,player6_pocket_card1,player6_pocket_card2,player6_name,player7_total_bet,player7_bankroll,player7_total_win,player7_bet_preflop,player7_bet_flop,player7_bet_turn,player7_bet_river,player7_pocket_card1,player7_pocket_card2,player7_name,player8_total_bet,player8_bankroll,player8_total_win,player8_bet_preflop,player8_bet_flop,player8_bet_turn,player8_bet_river,player8_pocket_card1,player8_pocket_card2,player8_name,player9_total_bet,player9_bankroll,player9_total_win,player9_bet_preflop,player9_bet_flop,player9_bet_turn,player9_bet_river,player9_pocket_card1,player9_pocket_card2,player9_name,player10_total_bet,player10_bankroll,player10_total_win,player10_bet_preflop,player10_bet_flop,player10_bet_turn,player10_bet_river,player10_pocket_card1,player10_pocket_card2,player10_name,player11_total_bet,player11_bankroll,player11_total_win,player11_bet_preflop,player11_bet_flop,player11_bet_turn,player11_bet_river,player11_pocket_card1,player11_pocket_card2,player11_name,player12_total_bet,player12_bankroll,player12_total_win,player12_bet_preflop,player12_bet_flop,player12_bet_turn,player12_bet_river,player12_pocket_card1,player12_pocket_card2,player12_name
0,3,1,2,1995,5,800160769,-,-,-,-,-,25,8371,0,Bf,-,-,-,-,-,A8,50,8035,75,B,-,-,-,-,-,Schween,0,0,0,0,0,0,1,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,1,2,1995,5,800160788,Jc,Qs,6s,6d,-,350,8060,600,Bcc,kc,kr,-,-,-,Schween,250,8346,0,Br,b,bf,-,-,-,A8,2,200,2,300,0,0,1,600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,2,1995,5,800160827,9h,7s,Qc,-,-,150,8096,250,Br,b,-,-,-,-,A8,100,8310,0,Bc,f,-,-,-,-,Schween,2,200,0,0,0,0,1,250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

### EDA

In [4]:
df[df.board_card_4 != '-'].drop(columns=["board_card_5"])

,game_id,dealer_id,players_num,year,month,timestamp,board_card_1,board_card_2,board_card_3,board_card_4,player1_total_bet,player1_bankroll,player1_total_win,player1_bet_preflop,player1_bet_flop,player1_bet_turn,player1_bet_river,player1_pocket_card1,player1_pocket_card2,player1_name,player2_total_bet,player2_bankroll,player2_total_win,player2_bet_preflop,player2_bet_flop,player2_bet_turn,player2_bet_river,player2_pocket_card1,player2_pocket_card2,player2_name,pot_players_num_flop,pot_size_flop,pot_players_num_turn,pot_size_turn,pot_players_num_river,pot_size_river,pot_players_num_showdown,pot_size_showdown,player3_total_bet,player3_bankroll,player3_total_win,player3_bet_preflop,player3_bet_flop,player3_bet_turn,player3_bet_river,player3_pocket_card1,player3_pocket_card2,player3_name,player4_total_bet,player4_bankroll,player4_total_win,player4_bet_preflop,player4_bet_flop,player4_bet_turn,player4_bet_river,player4_pocket_card1,player4_pocket_card2,player4_name,player5_total_bet,player5_bankroll,player5_total_win,player5_bet_preflop,player5_bet_flop,player5_bet_turn,player5_bet_river,player5_pocket_card1,player5_pocket_card2,player5_name,player6_total_bet,player6_bankroll,player6_total_win,player6_bet_preflop,player6_bet_flop,player6_bet_turn,player6_bet_river,player6_pocket_card1,player6_pocket_card2,player6_name,player7_total_bet,player7_bankroll,player7_total_win,player7_bet_preflop,player7_bet_flop,player7_bet_turn,player7_bet_river,player7_pocket_card1,player7_pocket_card2,player7_name,player8_total_bet,player8_bankroll,player8_total_win,player8_bet_preflop,player8_bet_flop,player8_bet_turn,player8_bet_river,player8_pocket_card1,player8_pocket_card2,player8_name,player9_total_bet,player9_bankroll,player9_total_win,player9_bet_preflop,player9_bet_flop,player9_bet_turn,player9_bet_river,player9_pocket_card1,player9_pocket_card2,player9_name,player10_total_bet,player10_bankroll,player10_total_win,player10_bet_preflop,player10_bet_flop,player10_bet_turn,player10_bet_river,player10_pocket_card1,player10_pocket_card2,player10_name,player11_total_bet,player11_bankroll,player11_total_win,player11_bet_preflop,player11_bet_flop,player11_bet_turn,player11_bet_river,player11_pocket_card1,player11_pocket_card2,player11_name,player12_total_bet,player12_bankroll,player12_total_win,player12_bet_preflop,player12_bet_flop,player12_bet_turn,player12_bet_river,player12_pocket_card1,player12_pocket_card2,player12_name
1,3,1,2,1995,5,800160788,Jc,Qs,6s,6d,350,8060,600,Bcc,kc,kr,-,-,-,Schween,250,8346,0,Br,b,bf,-,-,-,A8,2,200,2,300,0,0,1,600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,1,2,1995,5,800160905,4s,7h,Tc,6h,250,7860,400,Br,b,b,-,-,-,Schween,150,8546,0,Bc,c,f,-,-,-,A8,2,200,2,300,0,0,1,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3,1,2,1995,5,800160933,3s,Jh,8c,2h,50,8396,100,Bc,k,k,k,2c,As,A8,50,8010,0,Bk,k,k,k,6h,9s,Schween,2,100,2,100,2,100,2,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [5]:
df.filter(regex=r"board_card_(\d+)")

,board_card_1,board_card_2,board_card_3,board_card_4,board_card_5
0,-,-,-,-,-
1,Jc,Qs,6s,6d,-
2,9h,7s,Qc,-,-
3,Qh,6s,3s,-,-
4,As,3c,6s,-,-
...,...,...,...,...,...
439677,-,-,-,-,-
439678,4h,Ts,2h,Qc,2d
439679,4d,3h,5d,5h,7c
439680,6h,9s,4d,Qh,6s


#### Pre-processing

**Todo**
- Extract the hands with the non-empty first 4 board cards ✅
- At each hand, the players should have their pocket cards available ✅

In [6]:
def extract_full_board_hands(df:pd.DataFrame):
    first_4_board_cards_df = df.filter(regex=r"board_card_(\d+)").drop(columns=["board_card_5"])
    full_board_df = df[~(first_4_board_cards_df == "-").any(axis=1)]
    return full_board_df

In [7]:
full_board_cards_df = extract_full_board_hands(df)
full_board_cards_df

,game_id,dealer_id,players_num,year,month,timestamp,board_card_1,board_card_2,board_card_3,board_card_4,board_card_5,player1_total_bet,player1_bankroll,player1_total_win,player1_bet_preflop,player1_bet_flop,player1_bet_turn,player1_bet_river,player1_pocket_card1,player1_pocket_card2,player1_name,player2_total_bet,player2_bankroll,player2_total_win,player2_bet_preflop,player2_bet_flop,player2_bet_turn,player2_bet_river,player2_pocket_card1,player2_pocket_card2,player2_name,pot_players_num_flop,pot_size_flop,pot_players_num_turn,pot_size_turn,pot_players_num_river,pot_size_river,pot_players_num_showdown,pot_size_showdown,player3_total_bet,player3_bankroll,player3_total_win,player3_bet_preflop,player3_bet_flop,player3_bet_turn,player3_bet_river,player3_pocket_card1,player3_pocket_card2,player3_name,player4_total_bet,player4_bankroll,player4_total_win,player4_bet_preflop,player4_bet_flop,player4_bet_turn,player4_bet_river,player4_pocket_card1,player4_pocket_card2,player4_name,player5_total_bet,player5_bankroll,player5_total_win,player5_bet_preflop,player5_bet_flop,player5_bet_turn,player5_bet_river,player5_pocket_card1,player5_pocket_card2,player5_name,player6_total_bet,player6_bankroll,player6_total_win,player6_bet_preflop,player6_bet_flop,player6_bet_turn,player6_bet_river,player6_pocket_card1,player6_pocket_card2,player6_name,player7_total_bet,player7_bankroll,player7_total_win,player7_bet_preflop,player7_bet_flop,player7_bet_turn,player7_bet_river,player7_pocket_card1,player7_pocket_card2,player7_name,player8_total_bet,player8_bankroll,player8_total_win,player8_bet_preflop,player8_bet_flop,player8_bet_turn,player8_bet_river,player8_pocket_card1,player8_pocket_card2,player8_name,player9_total_bet,player9_bankroll,player9_total_win,player9_bet_preflop,player9_bet_flop,player9_bet_turn,player9_bet_river,player9_pocket_card1,player9_pocket_card2,player9_name,player10_total_bet,player10_bankroll,player10_total_win,player10_bet_preflop,player10_bet_flop,player10_bet_turn,player10_bet_river,player10_pocket_card1,player10_pocket_card2,player10_name,player11_total_bet,player11_bankroll,player11_total_win,player11_bet_preflop,player11_bet_flop,player11_bet_turn,player11_bet_river,player11_pocket_card1,player11_pocket_card2,player11_name,player12_total_bet,player12_bankroll,player12_total_win,player12_bet_preflop,player12_bet_flop,player12_bet_turn,player12_bet_river,player12_pocket_card1,player12_pocket_card2,player12_name
1,3,1,2,1995,5,800160788,Jc,Qs,6s,6d,-,350,8060,600,Bcc,kc,kr,-,-,-,Schween,250,8346,0,Br,b,bf,-,-,-,A8,2,200,2,300,0,0,1,600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,1,2,1995,5,800160905,4s,7h,Tc,6h,-,250,7860,400,Br,b,b,-,-,-,Schween,150,8546,0,Bc,c,f,-,-,-,A8,2,200,2,300,0,0,1,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3,1,2,1995,5,800160933,3s,Jh,8c,2h,Qc,50,8396,100,Bc,k,k,k,2c,As,A8,50,8010,0,Bk,k,k,k,6h,9s,Schween,2,100,2,100,2,100,2,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [8]:
first_hand = full_board_cards_df.iloc[0]
first_hand

game_id                     3
dealer_id                   1
players_num                 2
year                     1995
month                       5
                         ... 
player12_bet_turn         NaN
player12_bet_river        NaN
player12_pocket_card1     NaN
player12_pocket_card2     NaN
player12_name             NaN
Name: 1, Length: 139, dtype: object

In [9]:
players_num = first_hand["players_num"]
pattern = rf'player[1-{players_num}]_pocket_card[1-2]'
pattern = rf'player([1-9]|1[0-{str(players_num)[1]}])_pocket_card[1-2]' if players_num > 9 else pattern
matching_columns = [col for col in first_hand.index if re.match(pattern, col)]
matching_columns
first_hand[matching_columns].isin(['-']).any().item()

True

In [10]:
first_hand.index

Index(['game_id', 'dealer_id', 'players_num', 'year', 'month', 'timestamp',
       'board_card_1', 'board_card_2', 'board_card_3', 'board_card_4',
       ...
       'player12_total_bet', 'player12_bankroll', 'player12_total_win',
       'player12_bet_preflop', 'player12_bet_flop', 'player12_bet_turn',
       'player12_bet_river', 'player12_pocket_card1', 'player12_pocket_card2',
       'player12_name'],
      dtype='object', length=139)

In [12]:
def check_hand_missing_players_cards(hand:pd.Series):
    players_num = hand["players_num"]
    pattern = rf'player[1-{players_num}]_pocket_card[1-2]'
    pattern = (
        rf'player([1-9]|[1-9][0-{str(players_num)[1]}])_pocket_card[1-2]' 
        if players_num > 9 
        else pattern
    )
    matching_columns = [idx for idx in hand.index if re.match(pattern, idx)]
    return hand[matching_columns].isin(['-']).any().item()

In [13]:
check_hand_missing_players_cards(first_hand)

True

In [14]:
tqdm.pandas(desc='DataFrame Operation')

In [15]:
hands_with_missing_players_cards_mask = full_board_cards_df.progress_apply(check_hand_missing_players_cards, axis=1)


ataFrame Operation: 100%|████████████████████████████████████████████████████| 192743/192743 [01:39<00:00, 1929.40it/s]

In [16]:
hands_with_full_players_cards = full_board_cards_df[~hands_with_missing_players_cards_mask]
hands_with_full_players_cards

,game_id,dealer_id,players_num,year,month,timestamp,board_card_1,board_card_2,board_card_3,board_card_4,board_card_5,player1_total_bet,player1_bankroll,player1_total_win,player1_bet_preflop,player1_bet_flop,player1_bet_turn,player1_bet_river,player1_pocket_card1,player1_pocket_card2,player1_name,player2_total_bet,player2_bankroll,player2_total_win,player2_bet_preflop,player2_bet_flop,player2_bet_turn,player2_bet_river,player2_pocket_card1,player2_pocket_card2,player2_name,pot_players_num_flop,pot_size_flop,pot_players_num_turn,pot_size_turn,pot_players_num_river,pot_size_river,pot_players_num_showdown,pot_size_showdown,player3_total_bet,player3_bankroll,player3_total_win,player3_bet_preflop,player3_bet_flop,player3_bet_turn,player3_bet_river,player3_pocket_card1,player3_pocket_card2,player3_name,player4_total_bet,player4_bankroll,player4_total_win,player4_bet_preflop,player4_bet_flop,player4_bet_turn,player4_bet_river,player4_pocket_card1,player4_pocket_card2,player4_name,player5_total_bet,player5_bankroll,player5_total_win,player5_bet_preflop,player5_bet_flop,player5_bet_turn,player5_bet_river,player5_pocket_card1,player5_pocket_card2,player5_name,player6_total_bet,player6_bankroll,player6_total_win,player6_bet_preflop,player6_bet_flop,player6_bet_turn,player6_bet_river,player6_pocket_card1,player6_pocket_card2,player6_name,player7_total_bet,player7_bankroll,player7_total_win,player7_bet_preflop,player7_bet_flop,player7_bet_turn,player7_bet_river,player7_pocket_card1,player7_pocket_card2,player7_name,player8_total_bet,player8_bankroll,player8_total_win,player8_bet_preflop,player8_bet_flop,player8_bet_turn,player8_bet_river,player8_pocket_card1,player8_pocket_card2,player8_name,player9_total_bet,player9_bankroll,player9_total_win,player9_bet_preflop,player9_bet_flop,player9_bet_turn,player9_bet_river,player9_pocket_card1,player9_pocket_card2,player9_name,player10_total_bet,player10_bankroll,player10_total_win,player10_bet_preflop,player10_bet_flop,player10_bet_turn,player10_bet_river,player10_pocket_card1,player10_pocket_card2,player10_name,player11_total_bet,player11_bankroll,player11_total_win,player11_bet_preflop,player11_bet_flop,player11_bet_turn,player11_bet_river,player11_pocket_card1,player11_pocket_card2,player11_name,player12_total_bet,player12_bankroll,player12_total_win,player12_bet_preflop,player12_bet_flop,player12_bet_turn,player12_bet_river,player12_pocket_card1,player12_pocket_card2,player12_name
6,3,1,2,1995,5,800160933,3s,Jh,8c,2h,Qc,50,8396,100,Bc,k,k,k,2c,As,A8,50,8010,0,Bk,k,k,k,6h,9s,Schween,2,100,2,100,2,100,2,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3,1,2,1995,5,800160964,3h,Kc,Ah,Jd,Qd,400,7960,0,Bcc,bc,kc,kc,6c,Kd,Schween,400,8446,800,Br,r,b,b,Jh,Qs,A8,2,200,2,400,2,600,2,800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,3,1,2,1995,5,800161083,5s,3h,4c,2s,3s,400,8921,800,Brc,br,k,b,4d,3d,A8,400,7485,0,Br,rc,k,c,8c,8d,Schween,2,300,2,600,2,600,2,800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [17]:
first_full_hand = hands_with_full_players_cards.iloc[0]
first_full_hand

game_id                     3
dealer_id                   1
players_num                 2
year                     1995
month                       5
                         ... 
player12_bet_turn         NaN
player12_bet_river        NaN
player12_pocket_card1     NaN
player12_pocket_card2     NaN
player12_name             NaN
Name: 6, Length: 139, dtype: object

### Todo
- strength of each player's hand
- the average strength of the oponents' hands
- the aggression factor for each player
- number of actions
- win/loss status

In [18]:
pattern = r"board_card_[1-4]"
first_full_hand.filter(regex=pattern).values

array(['3s', 'Jh', '8c', '2h'], dtype=object)

In [19]:
evaluator = Evaluator()

In [20]:
board_cards = [Card.new(val) for val in first_full_hand.filter(regex=pattern).values]
Card.print_pretty_cards(board_cards)

 [3♠],[J♥],[8♣],[2♥] 


In [21]:
evaluator.evaluate(board_cards, [Card.new(c) for c in ["Qd", "Kh"]])

6697

In [22]:
pn = first_full_hand["players_num"]
for i in range(1, pn+1):
    player_name = first_full_hand[f"player{i}_name"]
    won_status = df_utils.hand_winning_test(first_full_hand, player_name)
    
    print(player_name.ljust(10), "\t", won_status)

A8         	 True
Schween    	 False


In [23]:
pn = first_full_hand["players_num"]

pattern = (
    rf"board_card_(\d+)|player([1-{pn}]\D+)" 
    if pn<=9
    else rf"board_card_(\d+)|player([1-9]\D.+|[1-9][0-{str(pn)[1]}].+)" 
)
for i in range(1, pn+1):
    print(first_full_hand.filter(regex=pattern).index.tolist())
    break
    print("*"*30)

['board_card_1', 'board_card_2', 'board_card_3', 'board_card_4', 'board_card_5', 'player1_total_bet', 'player1_bankroll', 'player1_total_win', 'player1_bet_preflop', 'player1_bet_flop', 'player1_bet_turn', 'player1_bet_river', 'player1_pocket_card1', 'player1_pocket_card2', 'player1_name', 'player2_total_bet', 'player2_bankroll', 'player2_total_win', 'player2_bet_preflop', 'player2_bet_flop', 'player2_bet_turn', 'player2_bet_river', 'player2_pocket_card1', 'player2_pocket_card2', 'player2_name']


In [24]:
def avg(data:list[Number]):
    return sum(data) / len(data)

In [25]:
pn = first_full_hand["players_num"]
# print(pn)
pattern = (
    rf"board_card_(\d+)|player([1-{pn}]\D+)" 
    if pn<=9
    else rf"board_card_(\d+)|player([1-9]\D.+|[1-9][0-{str(pn)[1]}].+)" 
)
for i in range(1, pn+1):
    player_name = first_full_hand[f"player{i}_name"]
    other_players_names = [first_full_hand[f"player{j}_name"] for j in range(1, pn+1) if j != i]
    player_aggression_factor = df_utils.calculate_hand_aggression_factor(first_full_hand, player_name)
    others_aggression_factors = [df_utils.calculate_hand_aggression_factor(first_full_hand, player) for player in other_players_names]
    opponents_aggression_factor = round(sum(others_aggression_factors) / len(other_players_names), 3)
    board_cards = first_full_hand.filter(regex=r"board_card_(\d)").drop(["board_card_5"]).values
    board_cards = [Card.new(card) for card in board_cards]
    player_pocket_cards = first_full_hand.filter(regex=rf"player{i}_pocket_card[1-2]").values
    player_pocket_cards = [Card.new(pocket_card) for pocket_card in player_pocket_cards]
    player_hand_rank = evaluator.evaluate(board_cards, player_pocket_cards)
    others_pocket_cards = [first_full_hand.filter(regex=rf"player{j}_pocket_card[1-2]").values.tolist() for j in range(1, pn+1) if j != i]
    others_pocket_cards = [[Card.new(pocket_card) for pocket_card in player_pocket_cards] for player_pocket_cards in others_pocket_cards]
    others_hands_ranks = [evaluator.evaluate(board_cards, hand) for hand in others_pocket_cards]

    print("player's hand rank: ", player_hand_rank)
    print("oponents hands rank avg: ", avg(others_hands_ranks))
    print("="*30)

player's hand rank:  5987
oponents hands rank avg:  7279.0
player's hand rank:  7279
oponents hands rank avg:  5987.0


In [26]:
def preprocess_hand(hand:pd.Series):
    players_number = hand["players_num"]
    processed_players = []
    for i in range(1, players_number+1):
        player_name = hand[f"player{i}_name"]
        other_players_names = [hand[f"player{j}_name"] for j in range(1, pn+1) if j != i]
        player_aggression_factor = df_utils.calculate_hand_aggression_factor(hand, player_name)
        others_aggression_factors = [df_utils.calculate_hand_aggression_factor(hand, player) for player in other_players_names]
        opponents_aggression_factor = round(sum(others_aggression_factors) / len(other_players_names), 3)

        player_bankroll = hand[f"player{i}_bankroll"]
        other_players_bankrolls = [hand[f"player{j}_bankroll"] for j in range(1, pn+1) if j != i]
        opponents_bankroll_avg = np.array(other_players_bankrolls).mean().round(3).item()
        actions_number = df_utils.count_player_actions_type(hand, player_name)
        
        board_cards = first_full_hand.filter(regex=r"board_card_(\d)").drop(["board_card_5"]).values
        board_cards = [Card.new(card) for card in board_cards]
        player_pocket_cards = hand.filter(regex=rf"player{i}_pocket_card[1-2]").values
        player_pocket_cards = [Card.new(pocket_card) for pocket_card in player_pocket_cards]
        player_hand_rank = evaluator.evaluate(board_cards, player_pocket_cards)
        others_pocket_cards = [first_full_hand.filter(regex=rf"player{j}_pocket_card[1-2]").values.tolist() for j in range(1, pn+1) if j != i]
        others_pocket_cards = [[Card.new(pocket_card) for pocket_card in player_pocket_cards] for player_pocket_cards in others_pocket_cards]
        others_hands_ranks = [evaluator.evaluate(board_cards, hand) for hand in others_pocket_cards]
        opponents_hands_ranks_avg = math.floor(avg([evaluator.evaluate(board_cards, hand) for hand in others_pocket_cards]))

        is_hand_won = df_utils.hand_winning_test(hand, player_name).item()
        
        index = [
            "player_bankroll",
            "opponents_bankroll_avg",
            "actions_number", 
            "player_aggression_factor", 
            "opponents_aggression_factor", 
            "player_hand_rank", 
            "opponents_hands_ranks_avg", 
            "win_status"
        ]
        player_series = pd.Series([
            player_bankroll,
            opponents_bankroll_avg,
            actions_number,
            player_aggression_factor, 
            opponents_aggression_factor,
            player_hand_rank,
            opponents_hands_ranks_avg,
            is_hand_won
        ], index=index)
        processed_players.append(player_series)
    return processed_players

In [27]:
preprocess_hand(first_full_hand)

[player_bankroll                  8396
 opponents_bankroll_avg         8010.0
 actions_number                      5
 player_aggression_factor          0.0
 opponents_aggression_factor       0.0
 player_hand_rank                 5987
 opponents_hands_ranks_avg        7279
 win_status                       True
 dtype: object,
 player_bankroll                  8010
 opponents_bankroll_avg         8396.0
 actions_number                      5
 player_aggression_factor          0.0
 opponents_aggression_factor       0.0
 player_hand_rank                 7279
 opponents_hands_ranks_avg        5987
 win_status                      False
 dtype: object]

In [28]:
processed_hands = hands_with_full_players_cards.progress_apply(preprocess_hand, axis=1)


ataFrame Operation: 100%|████████████████████████████████████████████████████████| 35306/35306 [12:17<00:00, 47.85it/s]

In [29]:
flatten_processed_hands = chain.from_iterable(processed_hands.values.tolist())
flatten_processed_hands

In [30]:
model_df = pd.DataFrame(flatten_processed_hands)

### Model Building

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

In [53]:
def preprocess_and_split_data(new_df):
    pd.set_option('future.no_silent_downcasting', True)
    X = new_df.drop(columns=["win_status"])
    y = new_df["win_status"].replace({True: 1, False: 0}).astype('int32')
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test

In [54]:
X_train_scaled, X_test_scaled, y_train, y_test = preprocess_and_split_data(model_df)

In [55]:
y_train

22971    0
26554    0
60496    1
63159    1
17030    0
        ..
37194    0
6265     1
54886    1
860      0
15795    0
Name: win_status, Length: 64511, dtype: int32

In [56]:
def train_lightgbm(X_train, X_test, y_train, y_test):
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    params = {
        "objective": "binary",  # Use "binary" for binary classification
        "metric": "binary_error",  # Evaluation metric
        "boosting_type": "gbdt",  # Gradient Boosting Decision Trees
        "num_leaves": 31,  # Maximum number of leaves in one tree
        "learning_rate": 0.05,  # Learning rate
        "feature_fraction": 0.8,  # Proportion of features used per iteration
        "early_stopping_round": 10,  # Stop training if no improvement
        "seed": 42,  # Seed for reproducibility
    }

    model = lgb.train(
        params,
        train_data,
        num_boost_round=100,
        valid_sets=[train_data, test_data],
        valid_names=["train", "test"]
    )

    y_pred_prob = model.predict(X_test, num_iteration=model.best_iteration)
    y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    return model

In [57]:
model = train_lightgbm(X_train_scaled, X_test_scaled, y_train, y_test)

[LightGBM] [Info] Number of positive: 33284, number of negative: 31227
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005786 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 743
[LightGBM] [Info] Number of data points in the train set: 64511, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.515943 -> initscore=0.063794
[LightGBM] [Info] Start training from score 0.063794
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[12]	train's binary_error: 0.398428	test's binary_error: 0.397461
Accuracy: 0.6025

Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.45      0.52      3454
           1       0.59      0.74      0.66      3714

    accuracy                           0.60      7168
   macro avg       0.61  